In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pickle

# from module_.validation import validation
from module_.dataLoader import dataLoader
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection

In [ ]:
episodes, transitions, labels = dataLoader("hh101")

# np.save("./dataload/hh101/episodes.npy", episodes)
# np.save("./dataload/hh101/transitions.npy", transitions)
# np.save("./dataload/hh101/labels.npy", labels)

episodes = np.load("./dataload/hh101/episodes.npy", allow_pickle=True)
transitions = np.load("./dataload/hh101/transitions.npy")
labels = np.load("./dataload/hh101/labels.npy")

In [ ]:
test_index = 0
episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]

sensor_list = sorted(set(episode[:,0]))
segmentation_features = np.array(feature_extraction(episode, "hh101", sensor_list))

score_s = np.array(change_point_detection(segmentation_features, "RuLSIF"))

fig = plt.figure(figsize=(20, 1))
ax = fig.add_subplot(1, 1, 1)

plt.title(label)
plt.ylim(0, 1)
plt.plot(range(len(score_s)), score_s)
plt.axvline(x=transition, color="g", linestyle=":")
plt.axhline(y=0.1, color="r", linestyle=":")


In [ ]:
score_r = np.array(change_point_detection(segmentation_features, "RuLSIF"))
score_r[score_r<0]=0
fig = plt.figure(figsize=(20, 1))
ax = fig.add_subplot(1, 1, 1)

plt.title(label)
plt.ylim(0, 1)
plt.plot(range(len(score_r)), score_r)
plt.axvline(x=transition, color="g", linestyle=":")
plt.axhline(y=0.1, color="r", linestyle=":")

In [ ]:
label_list = set()
for label in labels:
    left, right = label.split("-")
    label_list.add(left); label_list.add(right)
label_list = sorted(label_list)
print(label_list)
events_dict = {label:(0, 0) for label in label_list}
sensor_dict = {label:(0, 0) for label in label_list}
duration_dict = {label:(0, 0) for label in label_list}
alarm_dict = {label:(0, 0) for label in label_list}

# scores = []
# scores_r = []
for ei in range(len(episodes)):
    if ei%100==0:
        print(ei, len(episodes))
    test_index = ei
    episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]

    # left
    leftlabel = label.split("-")[0]
    events_dict[leftlabel]=(events_dict[leftlabel][0]+len(episode[:transition]),events_dict[leftlabel][1]+1)
    sensor_dict[leftlabel]=(sensor_dict[leftlabel][0]+len(set(episode[:transition,0])),sensor_dict[leftlabel][1]+1)
    duration_dict[leftlabel]=(duration_dict[leftlabel][0]+float(episode[transition-1,2])-float(episode[0,2]),duration_dict[leftlabel][1]+1)

    # sensor_list = sorted(set(episode[:,0]))
    # segmentation_features = np.array(feature_extraction(episode, "hh101", sensor_list))

    # scores = np.array(change_point_detection(segmentation_features[:transition+2], "RuLSIF"))

    # alarm_dict[leftlabel]=(alarm_dict[leftlabel][0]+sum([1 for i in range(len(scores[:transition])) if scores[i]>0.1]),alarm_dict[leftlabel][1]+1)

# np.save("./replication/hh101/scores_s.npy", scores_s)
# np.save("./replication/hh101/scores_r.npy", scores_r)



In [9]:
for k, v in duration_dict.items():
    print(k, v[0]/v[1])

Bathe 971.6867308701787
Bed_Toilet_Transition 315.5651008942548
Cook 367.86838523757376
Eat 868.3863929993765
Enter_Home 15.455581864096084
Leave_Home 15.566632093191147
Other 113.9045015348399
Personal_Hygiene 199.44883613506622
Relax 4122.599046514428
Sleep 18251.816145040488
Wash_Dishes 223.19730555443536
Work 427.3478720982869


In [ ]:
# scores_s = np.load("./replication/hh101/scores_s.npy", allow_pickle=True)
scores_r = np.load("./replication/hh101/scores_r.npy", allow_pickle=True)

In [ ]:
true_positive = true_negative = false_positive = false_negative = 0

threshold = 0.1

for ei in range(len(episodes)):
    test_index = ei
    episode, transition, label = episodes[test_index], transitions[test_index], labels[test_index]
    # score_s = scores_s[test_index]
    score_r = scores_r[test_index]

    gt_timestamp_l = float(episode[transition-1][2])
    gt_timestamp_r = float(episode[transition][2])

    tp = tn = fp = fn =0
    activation = False
    activation_timestamp = 0
    for j in range(len(episode)):
        r = score_r[j]

        score = r
        timestamp = float(episode[j][2])

        if score>=threshold: #POSITIVE
            # if activation and timestamp - activation_timestamp <=20 : #SKIP
            #     if j!=transition:
            #         tn+=1
            #     else:
            #         fn+=1
            #     activation = False
            #     continue
            if j==transition:
                tp+=1
            else:
                if abs(gt_timestamp_l-timestamp)<=10 or abs(gt_timestamp_r-timestamp)<=10:
                    tp+=1
                else:
                    fp+=1
            activation = True
            activation_timestamp = timestamp
        else:
            if j!=transition:
                tn+=1
            else:
                fn+=1

    true_positive+=tp
    true_negative+=tn
    false_positive+=fp
    false_negative+=fn

In [ ]:
true_positive / (true_positive+false_negative), false_positive / (false_positive+true_negative)